In [1]:
import requests
import json
import sys
from datetime import datetime, date

In [2]:
BASE_URL = 'https://oscar.wmo.int/surface/rest/api'
SEARCH_API = BASE_URL + '/search/station?'
STATION_DETAILS = BASE_URL + '/stations/station/'

In [3]:
# we search for Romanian stations
arg = "territoryName=ROU&facilityType=landFixed"

In [4]:
r  = requests.get(SEARCH_API + arg)
if r.status_code != 200:
    print("ERROR: status: {}".format(r.status_code))
    sys.exit(1)

In [5]:
result = r.json()
print("we have {} results".format(len(result)))

we have 167 results


In [6]:
# no modification date
result[0]

{'id': 3656,
 'name': 'ADAMCLISI',
 'region': 'Europe',
 'territory': 'Romania',
 'declaredStatus': 'Operational',
 'latitude': 44.0882700719,
 'longitude': 27.965626028,
 'elevation': 158,
 'stationTypeName': 'Land (fixed)',
 'wigosStationIdentifiers': [{'wigosStationIdentifier': '0-20000-0-15479',
   'primary': True}],
 'wigosId': '0-20000-0-15479',
 'stationTypeId': 1,
 'dateEstablished': '1956-07-31T23:00:00.000+00:00',
 'stationStatusCode': 'operational',
 'stationTypeCode': 'landOceanSurface',
 'stationProgramsDeclaredStatuses': 'GOS:Operational'}

In [7]:
# loop over result and get the station details for each station
station_details = {}
count=0
for station in result:
    internal_id = station["id"]
    r = requests.get(STATION_DETAILS + "{internal_id}/stationReport".format(internal_id=internal_id))
    
    if r.status_code == 200:
        station_details[str(internal_id)] = r.json()
        print("downloaded {internal_id}".format(internal_id=internal_id))
    else:
        print("warning.. could not load station with id {internal_id}".format(internal_id=internal_id))
        
    if count>10:
        break
        
    count=count+1

downloaded 3656
downloaded 3571
downloaded 3567
downloaded 3659
downloaded 3614
downloaded 3564
downloaded 3549
downloaded 3637
downloaded 3511
downloaded 3623
downloaded 3652
downloaded 3553


In [8]:
station_details["3656"]

{'id': 3656,
 'name': 'ADAMCLISI',
 'typeName': 'Land (fixed)',
 'typeId': 1,
 'wmoRaName': 'VI - Europe',
 'wmoRaId': 6,
 'lastModifiedOn': '2020-06-10',
 'lastModifiedBy': '192e4000-13d2-4f48-a728-3991fb3e08cb',
 'lastModifiedByFullName': 'MANEA Ancuta',
 'aliases': [],
 'descriptions': [{'id': 6498,
   'description': 'Observation remarks: AGRIMET;AUT;M/B;PH;SOILTEMP;SUNDUR (see code table A for explanations). These remarks imply the following additional observations that could not be registered automatically: Station making reports of sudden changes; Phenological observations.',
   'validSince': '2016-04-28',
   'xmlId': 'id_b074e824-c735-7937-e053-148ef98d6de4'}],
 'timezones': [{'id': 1154,
   'timezoneName': 'UTC+2',
   'timezoneId': 3,
   'validSince': '1956-08-01'}],
 'organizations': [{'id': 5729,
   'organizationName': 'NMAR',
   'organizationId': 223,
   'validSince': '2016-04-28',
   'xmlId': 'id_b074e824-254c-7937-e053-148ef98d6de4'}],
 'topographies': [],
 'territories': 

In [11]:
# dictionary of internal_id and modification date
stations_modified = { internal_id: datetime.strptime( station['lastModifiedOn'],"%Y-%M-%d")  for internal_id,station in station_details.items()   }
# filter stations modified since 2019
my_stations = [ internal_id for internal_id,moddate in stations_modified.items() if moddate > datetime(2019,1,1)   ]

In [12]:
my_stations

['3656',
 '3571',
 '3567',
 '3659',
 '3614',
 '3564',
 '3549',
 '3637',
 '3511',
 '3623',
 '3652',
 '3553']

In [13]:
# print our result: name and modification date
for idx in my_stations:
    name=station_details[idx]['name']
    print("{name} was modified on {moddate}".format(name=name,moddate=stations_modified[idx]))
    

ADAMCLISI was modified on 2020-01-10 00:06:00
ADJUD was modified on 2020-01-16 00:07:00
ALBA IULIA was modified on 2020-01-07 00:07:00
ALEXANDRIA was modified on 2020-01-30 00:06:00
APA NEAGRA was modified on 2020-01-12 00:06:00
ARAD was modified on 2020-01-24 00:08:00
BACAU was modified on 2020-01-24 00:08:00
BACLES was modified on 2020-01-11 00:06:00
BAIA MARE was modified on 2020-01-07 00:07:00
BAILE HERCULANE was modified on 2020-01-31 00:07:00
BAILESTI was modified on 2020-01-12 00:06:00
BAISOARA was modified on 2020-01-08 00:07:00
